# Data preprocessing
## Preprocessing Airfare dataset
The data is loaded in tictactoe dataframe

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split

colnames=['top-left-square','top-middle-square','top-right-square','middle-left-square','middle-middle-square','middle-right-square','bottom-left-square','bottom-middle-square','bottom-right-square','Class']
filename = r"E:\Documents\University of Hildesheim\Machine learning lab\lab4\tic-tac-toe.data"
tictactoe = pd.read_csv(filename,delimiter=',',header=None,names=colnames)
tictactoe.head()


KeyboardInterrupt



# Data Analysis
### The dataset has no missing values and all the columns with object values, these could be treated as categorical.

In [ ]:
tictactoe.info()

In [ ]:
tictactoe = tictactoe.astype('category')

What can be experimented with is a simple categorical encoding, wherein each unique entry is assigned it's own number. Pandas does with relative ease by assigning desired object columns to a category dtype

In [ ]:
tictactoe.dtypes

In [ ]:
tictactoe=tictactoe.apply(lambda x: x.cat.codes if x.dtype.name == 'category' else x)

In [ ]:
tictactoe.dtypes

### It is shown that each categorical column has its unique values

In [ ]:
tictactoe.head()

### By counting the number of positive and negative we can see that the datasets has more number of postive than negative values hence it is unbalanced

In [ ]:
tictactoe['Class'].value_counts()

### The solution to make the dataset balanced is using stratified sampling.Stratified sampling is a sampling method where the researcher divides the population into separate groups which is called strata then a probability sample is drawn from each group.

In [ ]:
def stratify_sample(df, col, samples):
    n = min(samples, df[col].value_counts().min())
    df_ = df.groupby(col).apply(lambda x: x.sample(n))
    df_.index = df_.index.droplevel(0)
    return df_

In [ ]:
tictactoe_stratified=stratify_sample(tictactoe, 'Class', 1000)

### After stratified sampling the number of positive and negative is equal which makes the dataset balanced

In [ ]:
tictactoe_stratified['Class'].value_counts()

In [ ]:
tictactoe_stratified.head()

In [ ]:
features=['top-left-square','top-middle-square','top-right-square','middle-left-square','middle-middle-square','middle-right-square','bottom-left-square','bottom-middle-square','bottom-right-square']
Xdata = tictactoe_stratified[features]
Ydata = tictactoe_stratified['Class']

x_train, x_test, y_train, y_test =train_test_split(Xdata,Ydata,train_size=0.8,test_size=0.2,random_state=0)

print('x_train :',x_train.shape)
print('x_test :',x_test.shape)
print('y_train :',y_train.shape)
print('y_test :',y_test.shape)

### Reshaping the data

In [ ]:
y_train=y_train.reshape(-1,1)
y_test=y_train.reshape(-1,1)

# Logistic Regression

### Logistic Regression is a method in machine learning for classification problems to output discrete values
### Step length bolddriver algorithm function
The Bold Driver Heuristic makes the assumption that smaller step sizes are needed when closer to the optimum.It adjusts the step size based on the value of f (x) at time t.If the value of f (x) grows, the step size must decrease.If the value of f (x) decreases, the step size can be larger for faster convergence.

In [ ]:
def bolddriver(x_train,y_train,beta_hat,lr,mhuplus = 1.1, mhuminus = 0.5):
    
    lr = lr*mhuplus
    contiter = True
    iterations = 0
    
    y_hat=logistic_function(x_train,beta_hat) # The current predicted value of Y
    l_old=log_likelihood(x_train, y_train, beta_hat) 
    
    while contiter:

        grad_beta = (np.dot(np.transpose(x_train), y_train - y_hat))
        beta_hat = beta_hat+learn_rate*(np.dot(np.transpose(x_train), y_train - y_hat))
        

        y_hat=logistic_function(x_train,beta_hat) # The current predicted value of Y
        l=log_likelihood(x_train, y_train, beta_hat)
        
        
        if l - l_old <= 0:
            lr = lr*mhuminus
            iterations += 1
        else:
            return lr
        
        if iterations == 250:
            break
    return lr

In [ ]:
def log_likelihood(x, y, beta):
    z = np.dot(x, beta)
    log = np.sum( y*z - np.log(1 + np.exp(z)) )
    return log

def gradient_ascent(X, h, y):
    return np.dot(X.T, y - h)

def logistic_function(X, beta):
    z = x_train.dot(beta)
    return 1 / (1 + np.exp(-z))   

logloss = lambda y,ypred: np.mean((y*np.log(ypred)+(1-y)*np.log(1-ypred)))
cost = lambda y,ypred: np.mean((y - ypred)**2)


# logistic regression using stochastic gradient decent

In [ ]:
m_train,n_features = x_train.shape

num_iter     = 1000
learn_rate   = 0.00001

beta_hat     = np.random.random(n_features).reshape(-1,1)
relative_l= []
relative_loss=[]
loglosstest=[]

y_hat=logistic_function(x_train,beta_hat)
l=0
l_old=log_likelihood(x_train, y_train, beta_hat)

chunk_size = 20

for i in range(num_iter):
    
        loss_old  = cost(y_train,y_hat)
    
        for chunk in range(len(x_train)//chunk_size):
            
            x_chunk = x_train[chunk*chunk_size:min((chunk+1)*chunk_size,len(x_train))]
            y_chunk = y_train[chunk*chunk_size:min((chunk+1)*chunk_size,len(y_train))]
            
            y_hat=logistic_function(x_chunk,beta_hat)
            
            beta_hat = beta_hat+learn_rate*(np.dot(np.transpose(x_chunk), y_chunk - y_hat))
        
    
        
    
        learn_rate =  bolddriver(x_train,y_train,beta_hat,learn_rate,mhuplus = 1.1, mhuminus = 0.5)
    
        y_hat=logistic_function(x_train,beta_hat)
    
        loss_new  = cost(y_train,y_hat)
    
    
        l_old=l
        l=log_likelihood(x_train, y_train, beta_hat)

        relative_l.append(l-l_old)
        relative_loss.append(np.abs(loss_new.values-loss_old.values))
        loglosstest.append(logloss(y_test,y_hat))

        if i % 5 == 0:
            print(f"epochs: {i} log_likelihood: {(l-l_old)} learning rate: {learn_rate} loss:{np.abs(loss_new.values-loss_old.values)} Logloss : {logloss(y_test,y_hat).values}")

        last_epoch=i+1

        if np.abs(l-l_old) == 0:
            break    

In [ ]:
plt.xlabel("epochs")
plt.ylabel("log_likelihood")
plt.plot( np.arange(last_epoch),relative_l,'r')

In [ ]:
plt.xlabel("epochs")
plt.ylabel("differential cost")
plt.plot( np.arange(last_epoch),relative_loss,'r')

In [ ]:
plt.xlabel("epochs")
plt.ylabel("logloss")
plt.plot( np.arange(last_epoch),loglosstest,'r')

# Implement Newton Algorithm (learning rate)

Newton's method is a second-order optimization algorithm that can help us find the best weights in our logistic function in fewer iterations compared to batch gradient descent.

The generalization of Newton’s method to a multidimensional setting (also called the Newton-Raphson method) is given by:

Where the Hessian is represented by:

For Logistic Regression, the Hessian is given by:

$$
Hf(\beta) = -X^TWX
$$
and the gradient is:

$$
\nabla f(\beta) = X^T(y-p)
$$
where$$
W := \text{diag}\left(p(1-p)\right)
$$
and $p$ are the predicted probabilites computed at the current value of $\beta$.

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def newton(beta0, y, X, lr):
    
    p = np.array(sigmoid(X.dot(beta0[:,0]))).T  
    W = np.diag((p*(1-p))) 
    
    
    hess = np.dot((np.dot(X.T,W)),X)  
    grad = (np.transpose(X)).dot(y-p)  
     
    
    s =lr*(np.dot(np.linalg.inv(hess), grad))      
    beta = beta0 + s
    
    return beta

### Using the Newton method we noticed that the convergence is faster because it needed only 5 iteration to converge.

In [ ]:
num_iter = 100
 
lr =0.001

relative_l_newton=[]
relative_loss_newton=[]
loglosstest_newton=[]

beta_old, beta = np.ones((n_features,1)), np.zeros((n_features,1))
l_newton=0
y_hat_newton=logistic_function(x_train,beta_old)
l_old_newton=-log_likelihood(x_train, y_train, beta_old)

for i in range(num_iter):
    
    loss_old_newton  = cost(y_train,y_hat_newton)
    beta_old = beta
    
    beta = newton_step(beta, y_train, x_train, reg_term)
    y_hat_newton=logistic_function(x_train,beta)
    
    loss_new_newton  = cost(y_train,y_hat_newton)
    
    l_old_newton=l_newton
    l_newton=-log_likelihood(x_train, y_train, beta)
    relative_l_newton.append(l_newton-l_old_newton)
    relative_loss_newton.append(np.abs(loss_new_newton.values-loss_old_newton.values))
    
   
    loglosstest_newton.append(-logloss(y_test,y_hat_newton))
    
    if i % 1 == 0:
        print(f"epochs: {i} log_likelihood: {(l_newton-l_old_newton)} loss:{np.abs(loss_new_newton.values-loss_old_newton.values)}")
    
    
    
    if np.abs(l_newton-l_old_newton) == 0:
        break
    

In [ ]:
plt.xlabel("epochs")
plt.ylabel("log_likelihood")
plt.plot( np.arange(len(relative_l_newton)),relative_l_newton,'r')

In [ ]:
plt.xlabel("epochs")
plt.ylabel("cost")
plt.plot( np.arange(len(relative_loss_newton)),relative_loss_newton,'r')

In [ ]:
plt.xlabel("epochs")
plt.ylabel("logloss")
plt.plot( np.arange(len(loglosstest_newton)),loglosstest_newton,'r')